# Let's Build GPT
Building a character-prediction GPT model from scratch, following [Andrej Karpathy's tutorial](https://youtu.be/kCc8FmEb1nY).

### Data Loading

In [1]:
# Download the tiny shakespeare dataset
! wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-10-27 14:53:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-10-27 14:53:56 (30.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print(f'Length of dataset: {len(text):,} characters')

Length of dataset: 1,115,394 characters


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# Sorted, unique characters occuring in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
unique_chars = ''.join(chars)
print(f'Unique characters: {unique_chars}')
print(f'Vocab size: {vocab_size}')

Unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


In [30]:
# Create dictionaries mapping strings to integers, and integers to strings
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }

# Define lambda functions that convert between string and integer
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [31]:
# Very small codebooks result in longer sequence lengths
print(encode("Hello, ChatGPT!"))
print(decode(encode("Hello, ChatGPT!")))

[20, 43, 50, 50, 53, 6, 1, 15, 46, 39, 58, 19, 28, 32, 2]
Hello, ChatGPT!


In [7]:
# Tokenize the tiny shakespeare dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long) # Why use torch.long?
print(data.shape, data.dtype)

# This corresponds to the first 1000 characters of shakespeare printed above
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# Split data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
"""
Training in this way is computationally efficient and also
helps the transformer to predict the next token for input sizes ranging
from 1 to block_size
"""

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} target is {target}')

When input is tensor([18]) target is 47
When input is tensor([18, 47]) target is 56
When input is tensor([18, 47, 56]) target is 57
When input is tensor([18, 47, 56, 57]) target is 58
When input is tensor([18, 47, 56, 57, 58]) target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [13]:
"""
Batching allows us to stack chunks. They are processed independently,
but it allows us to take advantage of parallelization.
"""

torch.manual_seed(1337)
batch_size = 4        # Number of independent sequences to be processed in parallel
block_size = 8        # Context (sequence) length for prediction

def get_batch(split):
    data = train_data if split == 'train' else val_data
    
    # Generate batch_size (4) random indices in the data to stack along for x and y
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [16]:
xb, yb = get_batch('train')
print('inputs are randomly selected:')
print(xb.shape)
print(xb)
print('targets are offset by 1:')
print(yb.shape)
print(yb)

inputs are randomly selected:
torch.Size([4, 8])
tensor([[43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39]])
targets are offset by 1:
torch.Size([4, 8])
tensor([[ 1, 51, 39, 63,  1, 40, 43,  1],
        [46, 43,  1, 43, 39, 56, 57, 10],
        [58, 47, 53, 52, 12,  1, 37, 53],
        [56, 43,  1, 21,  1, 41, 39, 51]])


### Bigram Model

In [32]:
"""
We start with a simple bigram language model.
The bigram language model predicts the next token for a given token.
"""

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        
        # Each token reads logits for next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # idx and targets are tensors of (batch_size, time) or (batch_size, block_size)
        logits = self.token_embedding_table(idx) # (B,T,C) or (batch_size, block_size, vocab_size)
        
        if targets is None:
            loss = None
        else:
            # Negative log-likelihood is a good quality measure for predictions
            # PyTorch expects (B,C,T) instead of (B,T,C)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)        
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # Take last element (B,T,C) -> (B,C)
            probs = F.softmax(logits, dim=-1) # Get softmax probs
            idx_next = torch.multinomial(probs, num_samples=1) # Sample using probs -> (B,1)
            idx = torch.cat((idx, idx_next), dim=1) # Concat next token to integers
        return idx

In [33]:
# We can print logits and observe the shape
# The output are next-token logits for each token in the batch
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print("Shape of logits: ", logits.shape)
print("Loss: ", loss)

Shape of logits:  torch.Size([32, 65])
Loss:  tensor(4.6841, grad_fn=<NllLossBackward0>)


In [34]:
# Generate from the bigram model
gen_input = torch.zeros((1,1), dtype=torch.long)
gen_output = m.generate(gen_input, max_new_tokens=100)[0].tolist()
gen_decode = decode(gen_output)
print(gen_decode)


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [35]:
# Create a PyTorch optimizer (Adam is the most popular, larger LR is okay for simpler models)
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [40]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.631432294845581


In [42]:
# Outputs still nonsensical, but better than before
# Generate from the bigram model
gen_input = torch.zeros((1,1), dtype=torch.long)
gen_output = m.generate(gen_input, max_new_tokens=100)[0].tolist()
gen_decode = decode(gen_output)
print(gen_decode)


Brs hirgou s iger m t.
A: st t t mburme?
I We, Theagr.
LEROLYCYous te!
Whitaslkn hauceathesthad g he


### Self-Attention Tricks

In [25]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [24]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)

# Bag of words averages the features of previous tokens
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev,0)

In [4]:
# Initial tensor for batch 0
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [5]:
# Averaged tensor for batch 0
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [6]:
# A more efficient approach using matrix math
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0, 10, (3,2)).float()
c = a @ b

print("a: \n", a)
print("b: \n", b)
print("c: \n", c)

a: 
 tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b: 
 tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c: 
 tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [7]:
# A more efficient approach using matrix math
# The following tensor ops incrementally average over rows of b
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b

print("a: \n", a)
print("b: \n", b)
print("c: \n", c)

a: 
 tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b: 
 tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c: 
 tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [15]:
wei = torch.tril(torch.ones(T, T)) # (block_size, block_size)
wei = wei / wei.sum(1, keepdim=True)

# (B*,T,T) @ (B, T, C) -> (B,T,C)
# Matmul broadcasts over batch dim
xbow2 = wei @ x 

# Check that result is the same as using for loops
torch.allclose(xbow, xbow2)

True

In [21]:
# version 3
print("T: ", T)

T:  8


In [19]:
tril = torch.tril(torch.ones(T,T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [20]:
wei = torch.zeros((T,T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [22]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [26]:
# Softmax performs exponentiation and normalization
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])